# Loading and Cleaning the Toloka Data

In [ ]:
import pandas as pd
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving new_df2.csv to new_df2.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['new_df2.csv']))

In [ ]:
df.head()

,Unnamed: 0,title,project_id,adult_content,lang,description,instructions_flag,requester_id,requester_trusted_flag,project_stat,...,moneyMax3,pool_id,pool_started_at,acceptance_period,user_id,task_attempted_flag,country,user_language,birth_day,joined
0,0,[Toloka] English Grammar Test,105273,0.0,EN,Take the test and get more paid tasks.\nAfter ...,1.0,71daea4cffae4488067aebfb6583914a,0.0,4.75,...,NaN,36185755.0,2022-11-03T19:07:12.911,NaN,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
1,1,[Toloka] English Grammar Test,105273,0.0,EN,Take the test and get more paid tasks.\nAfter ...,1.0,71daea4cffae4488067aebfb6583914a,0.0,4.75,...,NaN,36185755.0,2022-11-03T19:07:13.000Z,NaN,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
2,2,[Toloka] Prueba de comprensión de español,62518,0.0,EN,Haz la prueba y recibe más tareas pagadas.\nSi...,0.0,71daea4cffae4488067aebfb6583914a,0.0,4.50,...,NaN,35994576.0,2022-10-22T06:38:00.000Z,NaN,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
3,3,Page relevance (new version),81714,0.0,NaN,Passing the exam with a good quality opens the...,1.0,0f3f3a1c8053bf6128f7a3b730e9162c,1.0,4.39,...,0.31,33608621.0,2022-05-27T08:40:05.000Z,NaN,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
4,4,Is the image a good picture of the zoo animal?,84339,1.0,EN,Please judge if the image is showing a correct...,1.0,e134d662550bd2d2176b4f3e515f6e23,0.0,4.57,...,NaN,32387125.0,2022-03-15T21:21:42.000Z,NaN,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z


In [ ]:
log_data = pd.read_csv(io.BytesIO(uploaded['log_data.csv']))

In [ ]:
log_data.head()

,Unnamed: 0,id,user_id,event,timestamp,url,tab_id,scroll_count,blur_count,focus_count,click_count,keypress_count
0,0,1,0,TABCLOSED,1668120340446,NaN,837345710,0.0,0.0,0.0,0.0,0.0
1,1,2,0,TABUPDATED,1667956551791,https://www.sammeechward.com/deploy-node-app-o...,837344770,NaN,NaN,NaN,NaN,NaN
2,2,3,0,TABUPDATED,1667956551791,NON-TOLOKA,837344770,NaN,NaN,NaN,NaN,NaN
3,3,4,1709755371,TABUPDATED,1668628323406,NON-TOLOKA,837345509,NaN,NaN,NaN,NaN,NaN
4,4,5,1709755371,TABUPDATED,1668628325893,NON-TOLOKA,837347829,NaN,NaN,NaN,NaN,NaN


# Cleaning the data without the tables combined on Assignment Data

## Finding NaN values

In [ ]:
df.shape

(85, 23)

In [ ]:
df.isnull().sum()

Unnamed: 0                 0
title                     12
project_id                 0
adult_content             12
lang                      58
description               15
instructions_flag         12
requester_id              12
requester_trusted_flag    12
project_stat              24
avg_money_hourly          19
moneyMed                  36
moneyTop10                36
moneyMax3                 58
pool_id                   12
pool_started_at           12
acceptance_period         72
user_id                   58
task_attempted_flag        0
country                   58
user_language             58
birth_day                 58
joined                    58
dtype: int64

## Imputing the missing values

* user_id: too many NaN values.
* Occuring from task_data since task_data doesn't have user_id column and to get that we are merging it with task_receipt 
* However; not all projects are present in the receipt so it is creating a lot of NaN values.

Imputing all the user data with mode

In [ ]:
df['user_id'] = df['user_id'].fillna(df['user_id'].mode()[0])

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

In [ ]:
df['user_language'] = df['user_language'].fillna(df['user_language'].mode()[0])

In [ ]:
df['birth_day'] = df['birth_day'].fillna(df['birth_day'].mode()[0])

In [ ]:
df['joined'] = df['joined'].fillna(df['joined'].mode()[0])

In [ ]:
df['lang'] = df['lang'].fillna(df['lang'].mode()[0])

**title, adult_content, instructions_flag, requester_id, requester_trusted_flag, pool_id, pool_started_at** : 
* Nan values in these columns are all the tasks that have their receipts generated but they're not present on the 'Tasks' tab anymore, therefore we couldn't fetch the above information for these tasks

For now. it's better to remove these 12 rows until we find a way to fetch task data for the tasks that are present in the task receipts but aren't present in task tab anymore.

In [ ]:
df = df[df['title'].notna()]

**avg_money_hourly, moneyMed, moneyTop10, moneyMax3**:
* Nan and other forms of missing values in these columns simply means that the requester hasn't provided any payment options. So we will replace them with 0. 

In [ ]:
df.avg_money_hourly.unique()

array([0.       , 0.371134 , 0.62531  , 0.108108 , 0.185567 ,       nan,
       0.28125  , 0.54     , 1.08897  , 0.8      , 0.791209 , 0.529412 ,
       2.0339   , 0.821918 , 0.657534 , 0.268657 , 0.576923 , 2.08092  ,
       0.514286 , 0.972973 , 0.633803 , 0.810811 , 0.475248 , 0.39779  ,
       0.743363 , 0.382979 , 0.444444 , 1.8      , 0.235294 , 0.0957445,
       0.452424 , 0.382166 , 0.96     , 0.507042 , 1.13089  ])

In [ ]:
df.moneyMed.unique()

array([ nan, 0.01, 0.  , 0.04, 0.02, 0.08, 0.1 , 0.03, 0.07, 0.35, 0.05,
       0.06, 0.18])

In [ ]:
df.moneyTop10.unique()

array([ nan, 0.04, 0.18, 0.69, 0.73, 0.01, 0.03, 0.36, 0.16, 0.3 , 0.  ,
       0.08, 0.1 , 0.21, 0.07, 0.5 , 0.12, 0.38, 0.96, 0.05, 0.09, 0.83,
       5.06, 0.06, 0.2 ])

In [ ]:
df.moneyMax3.unique()

array([      nan, 3.100e-01, 2.910e+00, 2.003e+01, 1.004e+01, 1.000e-02,
       1.300e-01, 2.100e+00, 1.260e+00, 1.900e+00, 1.070e+00, 1.100e+00,
       8.600e-01, 2.820e+00, 3.170e+00, 3.700e-01, 1.800e+00, 1.228e+01,
       4.800e-01])

In [ ]:
df['avg_money_hourly'] = df['avg_money_hourly'].fillna(0)
df['moneyMed'] = df['moneyMed'].fillna(0)
df['moneyMax3'] = df['moneyMax3'].fillna(0)
df['moneyTop10'] = df['moneyTop10'].fillna(0)

**acceptance_period**: 
* too many missing values(72), it makes more sense to drop the entire column than trying to impute these values

In [ ]:
df = df.drop('acceptance_period', axis = 1)

**project_stat**

In [ ]:
df.project_stat.unique()

array([4.75, 4.5 , 4.39, 4.57, 4.83, 4.33, 4.21, 4.67, 1.6 , 4.68, 4.  ,
       4.3 , 4.77,  nan, 4.72, 4.81, 4.45, 4.47, 4.69, 4.91, 4.82, 4.03,
       4.65, 4.66, 4.6 , 4.64, 4.48, 4.71, 4.93, 4.97, 4.26, 4.58, 4.95,
       3.11, 4.7 , 4.86, 4.61, 1.45])

project_stat is basically the grade assigned to that project. I am not sure what exactly is the criteria for that grade assignment. However, for now it makes sense to just replace unassigned grades with 0.

In [ ]:
df['project_stat'] = df['project_stat'].fillna(0)

In [ ]:
df.head()

,Unnamed: 0,title,project_id,adult_content,lang,description,instructions_flag,requester_id,requester_trusted_flag,project_stat,...,moneyTop10,moneyMax3,pool_id,pool_started_at,user_id,task_attempted_flag,country,user_language,birth_day,joined
0,0,[Toloka] English Grammar Test,105273,0.0,EN,Take the test and get more paid tasks.\nAfter ...,1.0,71daea4cffae4488067aebfb6583914a,0.0,4.75,...,0.00,0.00,36185755.0,2022-11-03T19:07:12.911,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
1,1,[Toloka] English Grammar Test,105273,0.0,EN,Take the test and get more paid tasks.\nAfter ...,1.0,71daea4cffae4488067aebfb6583914a,0.0,4.75,...,0.00,0.00,36185755.0,2022-11-03T19:07:13.000Z,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
2,2,[Toloka] Prueba de comprensión de español,62518,0.0,EN,Haz la prueba y recibe más tareas pagadas.\nSi...,0.0,71daea4cffae4488067aebfb6583914a,0.0,4.50,...,0.00,0.00,35994576.0,2022-10-22T06:38:00.000Z,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
3,3,Page relevance (new version),81714,0.0,EN,Passing the exam with a good quality opens the...,1.0,0f3f3a1c8053bf6128f7a3b730e9162c,1.0,4.39,...,0.04,0.31,33608621.0,2022-05-27T08:40:05.000Z,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z
4,4,Is the image a good picture of the zoo animal?,84339,1.0,EN,Please judge if the image is showing a correct...,1.0,e134d662550bd2d2176b4f3e515f6e23,0.0,4.57,...,0.00,0.00,32387125.0,2022-03-15T21:21:42.000Z,1.699066e+09,True,US,EN,2000-07-07T00:00:00.000Z,2022-10-11T00:00:00.000Z


In [ ]:
df.isnull().sum()

Unnamed: 0                0
title                     0
project_id                0
adult_content             0
lang                      0
description               3
instructions_flag         0
requester_id              0
requester_trusted_flag    0
project_stat              0
avg_money_hourly          0
moneyMed                  0
moneyTop10                0
moneyMax3                 0
pool_id                   0
pool_started_at           0
user_id                   0
task_attempted_flag       0
country                   0
user_language             0
birth_day                 0
joined                    0
dtype: int64

Now we are just left with 3 NaN values in 'description' column. We would not be needing description column at all unless we are planning to incorporate NLP. However, for now I will just be dropping these three rows so we still will be having the 'description' column in case we decide to implement NLP. Also we can later on probably impute these values using NLP as well.

In [ ]:
df = df[df['description'].notna()]

In [ ]:
df.isnull().sum()

Unnamed: 0                0
title                     0
project_id                0
adult_content             0
lang                      0
description               0
instructions_flag         0
requester_id              0
requester_trusted_flag    0
project_stat              0
avg_money_hourly          0
moneyMed                  0
moneyTop10                0
moneyMax3                 0
pool_id                   0
pool_started_at           0
user_id                   0
task_attempted_flag       0
country                   0
user_language             0
birth_day                 0
joined                    0
dtype: int64

In [ ]:
df.shape

(70, 22)

In [ ]:
df_new = df

Now the cleaned data contains 70 rows and 22 columns as compared to the raw data with 85 rows and 23 columns.

# Cleaning again with the tables combined on Assignment Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving uncleaned_data (2).csv to uncleaned_data (2).csv


In [ ]:
df2 = pd.read_csv(io.BytesIO(uploaded['uncleaned_data (2).csv']))

In [ ]:
df2.columns

Index(['Unnamed: 0', 'user_id', 'project_id', 'title', 'adult_content', 'lang',
       'description', 'instructions_flag', 'requester_id',
       'requester_trusted_flag', 'project_stat', 'avg_money_hourly',
       'moneyMed', 'moneyTop10', 'moneyMax3', 'pool_id', 'pool_started_at',
       'acceptance_period', 'task_attempted_flag', 'country', 'user_language',
       'birth_day', 'joined'],
      dtype='object')

In [ ]:
df2.isnull().sum()

Unnamed: 0                   0
user_id                    444
project_id                   2
title                      394
adult_content              394
lang                      5497
description                593
instructions_flag          394
requester_id               394
requester_trusted_flag     394
project_stat              1653
avg_money_hourly           576
moneyMed                  1939
moneyTop10                1939
moneyMax3                 4564
pool_id                    394
pool_started_at            394
acceptance_period         8852
task_attempted_flag        508
country                    444
user_language              444
birth_day                  444
joined                     444
dtype: int64

## Imputing the missing values

**user_id, user_language, birth_day, joined, country**
* there are 39 missing values form the user data, not sure where they are occuring from we can impute these values again using mode.

In [ ]:
df2['user_id'] = df2['user_id'].fillna(df2['user_id'].mode()[0])

In [ ]:
df2['country'] = df2['country'].fillna(df2['country'].mode()[0])

In [ ]:
df2['user_language'] = df2['user_language'].fillna(df2['user_language'].mode()[0])

In [ ]:
df2['birth_day'] = df2['birth_day'].fillna(df2['birth_day'].mode()[0])

In [ ]:
df2['joined'] = df2['joined'].fillna(df2['joined'].mode()[0])

**lang, acceptance_period**
* these two columns have many missing values and we can simply drop these columns as they aren't as important as well

In [ ]:
df2 = df2.drop('lang', axis = 1)

In [ ]:
df2 = df2.drop('acceptance_period', axis = 1)

**title, adult_content, instructions_flag, requester_id, requester_trusted_flag, pool_id, pool_started_at** : 
* Nan values in these columns are all the tasks that have their receipts generated but they're not present on the 'Tasks' tab anymore, therefore we couldn't fetch the above information for these tasks

For now. it's better to remove these 3 rows until we find a way to fetch task data for the tasks that are present in the task receipts but aren't present in task tab anymore.

In [ ]:
df2 = df2[df2['title'].notna()]

**avg_money_hourly, moneyMed, moneyTop10, moneyMax3**:
* Nan and other forms of missing values in these columns simply means that the requester hasn't provided any payment options. So we will replace them with 0. 

In [ ]:
df2['avg_money_hourly'] = df2['avg_money_hourly'].fillna(0)
df2['moneyMed'] = df2['moneyMed'].fillna(0)
df2['moneyMax3'] = df2['moneyMax3'].fillna(0)
df2['moneyTop10'] = df2['moneyTop10'].fillna(0)

Now we are just left with 3 NaN values in 'description' column. We would not be needing description column at all unless we are planning to incorporate NLP. However, for now I will just be dropping these three rows so we still will be having the 'description' column in case we decide to implement NLP. Also we can later on probably impute these values using NLP as well.

In [ ]:
df2 = df2[df2['description'].notna()]

**project_stat**

In [ ]:
df2.project_stat.unique()

array([4.92, 4.54, 4.76, 4.67, 4.64, 4.24, 4.23, 4.09, 4.08, 4.03, 4.2 ,
       4.26, 4.21,  nan, 3.15, 4.77, 4.73, 4.72, 4.85, 4.59, 4.68, 4.93,
       4.95, 4.05, 4.39, 4.46, 4.71, 4.74, 4.75, 4.49, 4.65, 4.51, 4.94,
       4.9 , 4.66, 4.63, 4.31, 4.56, 2.52, 2.06, 4.89, 4.79, 4.78, 4.86,
       4.61, 4.6 , 4.33, 3.75, 4.82, 4.8 , 4.83, 4.81, 4.96, 4.87, 4.84,
       4.7 , 4.62, 5.  , 4.91, 4.88, 4.99, 4.97, 1.26, 4.14, 4.98, 1.92,
       4.35, 4.69, 4.38, 3.11, 3.88, 1.39, 2.26, 2.42, 2.79, 4.53, 4.52,
       4.18, 4.48, 4.58, 3.32, 4.5 , 4.47, 4.36, 4.57, 4.34, 4.41, 4.02])

project_stat is basically the grade assigned to that project. I am not sure what exactly is the criteria for that grade assignment. However, for now it makes sense to just replace unassigned grades with 0.

In [ ]:
df2['project_stat'] = df2['project_stat'].fillna(0)

In [ ]:
df2.isnull().sum()

Unnamed: 0                  0
user_id                     0
project_id                  0
title                       0
adult_content               0
description                 0
instructions_flag           0
requester_id                0
requester_trusted_flag      0
project_stat                0
avg_money_hourly            0
moneyMed                    0
moneyTop10                  0
moneyMax3                   0
pool_id                     0
pool_started_at             0
task_attempted_flag       472
country                     0
user_language               0
birth_day                   0
joined                      0
dtype: int64

In [ ]:
df2 = df2.reset_index(drop=True)


In [ ]:
df3=df2

# Feature Engineering

* Creating 'age' column from 'birth_day' column

In [ ]:
from datetime import date

In [ ]:
def age(birthdate):
  today = date.today()
  age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
  return age

In [ ]:
df2['birth_day'][10]

'1984-03-11T00:00:00.000Z'

In [ ]:
df2['birth_day'] = df2['birth_day'].astype(str).str[:10]

In [ ]:
df2['birth_day'] = pd.to_datetime(df2['birth_day'], errors='coerce')

In [ ]:
age_list = []
for i in range(len(df2['birth_day'])):
  ages = age(df2['birth_day'][i])
  age_list.append(ages)

In [ ]:
df2['age'] = age_list

In [ ]:
df2.head()

,Unnamed: 0,user_id,project_id,title,adult_content,description,instructions_flag,requester_id,requester_trusted_flag,project_stat,...,moneyTop10,moneyMax3,pool_id,pool_started_at,task_attempted_flag,country,user_language,birth_day,joined,age
0,0,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
1,1,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
2,2,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
3,3,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
4,4,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38


In [ ]:
print(df2.columns)

Index(['Unnamed: 0', 'user_id', 'project_id', 'title', 'adult_content',
       'description', 'instructions_flag', 'requester_id',
       'requester_trusted_flag', 'project_stat', 'avg_money_hourly',
       'moneyMed', 'moneyTop10', 'moneyMax3', 'pool_id', 'pool_started_at',
       'task_attempted_flag', 'country', 'user_language', 'birth_day',
       'joined', 'age'],
      dtype='object')


In [ ]:
df2.shape

(9770, 22)

In [ ]:
df2.head()

,Unnamed: 0,user_id,project_id,title,adult_content,description,instructions_flag,requester_id,requester_trusted_flag,project_stat,...,moneyTop10,moneyMax3,pool_id,pool_started_at,task_attempted_flag,country,user_language,birth_day,joined,age
0,0,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
1,1,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
2,2,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
3,3,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
4,4,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38


In [ ]:
df3 = df2.iloc[: , 1:]

In [ ]:
df3.head()

,user_id,project_id,title,adult_content,description,instructions_flag,requester_id,requester_trusted_flag,project_stat,avg_money_hourly,...,moneyTop10,moneyMax3,pool_id,pool_started_at,task_attempted_flag,country,user_language,birth_day,joined,age
0,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,2.14286,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
1,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,2.14286,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
2,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,2.14286,...,0.06,0.0,36770631.0,2022-12-09T15:07:41.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
3,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,2.16867,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38
4,1.316041e+09,49390.0,Entity image selection,1,From the selection of images for the entity(pe...,1,81ae1299dd10f764a6e59ac1c08a8a18,0,4.92,2.16867,...,0.06,0.0,36775179.0,2022-12-09T22:25:32.000Z,True,EG,"AR,EN,",1984-03-11,2021-01-13T00:00:00.000Z,38


In [ ]:
df_csv = df3.to_csv('cleaned_data.csv')